In [1]:
import kenlm
import sentencepiece
import unicodedata

sp_path="../data/lm_sp/ja.sp.model"
arpa_path="../data/lm_sp/ja.arpa.bin"

ken_model = kenlm.LanguageModel(arpa_path)

sp = sentencepiece.SentencePieceProcessor()
sp.load(sp_path)



True

In [2]:

inputs = ['東京はッ晴れ',
    '東京は元気です',
    '吾輩は猫である. 名前はまだない.',
    '東京は晴れ']

def calc_perplexity(inp, sp, ken_model):
    text = unicodedata.normalize('NFD', inp)
    toks = sp.encode(text, out_type=str)
    sentence = " ".join(toks)
    ppl = ken_model.perplexity(sentence)
    return ppl
for inp in inputs:
    # pretrained model in cc_net uses 'NFD' normalization?
    # TODO: Use https://github.com/facebookresearch/cc_net/blob/main/cc_net/text_normalizer.py
    text = unicodedata.normalize('NFD', inp)

    toks = sp.encode(text, out_type=str)
    print(toks)

    sentence = " ".join(toks)
    ppl = ken_model.perplexity(sentence)
    print(ppl, inp)

['▁東京', 'は', 'ッ', '晴れ']
10808.575564708846 東京はッ晴れ
['▁東京', 'は', '元気', 'で', 'す']
14207.90466675276 東京は元気です
['▁', '吾', '輩', 'は', '猫', 'で', 'ある', '.', '▁', '名前は', 'ま', 'だ', 'ない', '.']
677.481230499526 吾輩は猫である. 名前はまだない.
['▁東京', 'は', '晴れ']
3340.487952615284 東京は晴れ


In [3]:
from datasets import load_dataset


# ignore_verifications=Trueをつけないとエラーとなる
oscar_dataset = load_dataset('oscar', 'unshuffled_original_ja', 
                       split='train', 
                       ignore_verifications=True,
                       streaming=True)
#mc4の場合はこちら
mc4_dataset = load_dataset('mc4', 'ja',split='train', streaming=True)
dataset=oscar_dataset

/home/user/miniconda3/envs/crawl/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/user/miniconda3/envs/crawl/lib/python3.12/site-packages/datasets/load.py:2516: FutureWarning: 'ignore_verifications' was deprecated in favor of 'verification_mode' in version 2.9.1 and will be removed in 3.0.0.
You can remove this warning by passing 'verification_mode=no_checks' instead.
  warnings.warn(
/home/user/miniconda3/envs/crawl/lib/python3.12/site-packages/datasets/load.py:1454: FutureWarning: The repository for oscar contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/oscar
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this

In [4]:
loc_records = []
cnt=0
for record in dataset:
    loc_records.append(record)
    cnt+=1
    if cnt>10**2:
        break

In [5]:
import sys
sys.path.append('../src')
from cleaner import auto_filter

In [6]:
print(auto_filter.do_filter(text))

None


In [8]:
for record in loc_records:
    text=record['text']
    text=auto_filter.do_filter(text)
    if text is None:
        continue
    for line in text.split("\n"):
        ppl=calc_perplexity(line, sp, ken_model)
        print(ppl, line)

8797.823407496267 神社などへ一緒に同行して、様々な角度のショットで家族写真やお子様の写真を撮影致します！お好みに合わせて様々な写真を取ることができますので、その場でカメラマンへのリクエストも可能です！お子様の晴れ姿を、緊張していない自然な笑顔で残しませんか？
11615.869139023187 ※七五三の対象者が1名のプランです。ご兄弟、ご友人様もご一緒になどと、主役が2名、3名と複数名いらっしゃる場合は、＋5,000円（税抜）/1名の別途料金が発生いたします。（納品枚数＋約15カット、撮影時間＋20分/1名）
14595.047833691142 ※撮影神社、宮社によっては撮影許可料（3,000円ほど）が別途発生いたす場所がございます。こちらは弊社撮影料金には含まれておりませんので、事前に撮影希望神社へお客様よりご確認、お支払いをお願いしております。ご確認の程よろしくお願いいたします。
20766.861708389093 ◎正当な理由による書き込みの削除について：1レス￥５０００円　1スレ￥２００００円の技術作業料が発生します。一回分だけの料金で当方管理下の全サイトで作業が実施されます。支払い方法はAmazonギフト券番号。連絡先は当サイトの登録emailへ。
16825.98217933233 全国各地の歴史を物語る城などの史跡、東京タワーやスカイツリーなどの展望ができる観光スポット、雄大な景色が素晴らしい景勝地、日本の夏の風物詩である花火大会、冬の街を彩るイルミネーションスポット、イチゴ狩り・ブルーベリ狩り・なし狩り・ぶどう狩り・桃狩り・さくらんぼ狩りなどの季節の果物狩りが楽しめる農園、大人の工場見学にぴったりのワイナリー・ビール工場・酒蔵・マイクロブルワリー・蒸留所や、全国各地にちらばる観光スポットや観光名所を中心にご紹介します。おでかけ先が決まったあとも、お目当てのスポットとは違った視点での、施設や遊び方を「その他レジャー」で探してみてはいかがでしょうか。
26353.55937397255 秋のレジャーと言えばやはり紅葉狩り。素晴らしい紅葉を堪能できるスポットを紹介。もみじやカエデ、イチョウ。優雅に秋の色に包まれよう。昼には溢れんばかりに一面に広がる鮮やかなカラーを、夜にはライトアップさ
7479.713612715156 生牡蠣・